In [2]:
!pip install numpy
# Команда устанавливает библиотеку NumPy — основную библиотеку Python для работы с массивами, векторами, матрицами и научными вычислениями.

!pip install imutils
# Устанавливает


In [5]:
import numpy as np                     # Библиотека для работы с массивами и числовыми вычислениями
import matplotlib.pyplot as plt        # Библиотека для построения графиков
import cv2                             # OpenCV — для работы с изображениями
import os                              # Работа с файловой системой
from imutils import paths              # Получение списка путей к изображениям из папки
from tqdm import tqdm                  # Красивый прогресс-бар в циклах
from PIL import Image                  # Работа с изображениями через Pillow
import matplotlib.pyplot as plt        # Повторный импорт (избыточный)
from sklearn.cluster import KMeans     # Метод кластеризации (не используется в коде ниже)


In [3]:

# Функция для расчета гистограммы яркости изображения
def compute_histogram(image_path):
    image = Image.open(image_path).convert("L")
    # Открываем изображение и конвертируем его в оттенки серого ('L' = grayscale)

    image_np = np.array(image)
    # Преобразуем изображение в массив NumPy для последующей обработки

    hist = cv2.calcHist([image_np], [0], None, [256], [0, 256]).flatten()
    # Вычисляем гистограмму яркости:
    # [image_np] — входное изображение
    # [0] — используем только 1 канал (grayscale)
    # None — нет маски
    # [256] — число интервалов (bins) — от 0 до 255
    # [0, 256] — диапазон значений яркости
    # flatten() — преобразует результат в одномерный массив

    return hist / np.sum(hist)
    # Нормализуем гистограмму: сумма всех значений будет равна 1

# Функция для визуализации гистограммы
def plot_histogram(hist):
    plt.figure(figsize=(10, 5))
    # Устанавливаем размер всего рисунка

    plt.subplot(1, 2, 1)
    # Первая подграфика — линейная шкала
    plt.bar(range(256), hist, color="blue", alpha=0.7, width=1.0)
    # Столбчатая диаграмма: 256 значений от 0 до 255
    plt.title("Линейная шкала")
    plt.xlabel("Интенсивность")
    plt.ylabel("Частота")

    log_hist = np.log10(hist + 0.00001)
    # Переводим значения в логарифмическую шкалу, избегая log(0)

    log_hist = log_hist.max() - log_hist
    # Инвертируем логарифмы: более яркие — ниже

    plt.subplot(1, 2, 2)
    # Вторая подграфика — логарифмическая шкала
    plt.bar(range(256), log_hist, color="red", alpha=0.7, width=1.0)
    plt.title("Логарифмическая шкала")
    plt.xlabel("Интенсивность")
    plt.ylabel("log(Частота)")
    plt.tight_layout()
    # Метод библиотеки matplotlib, который автоматически настраивает отступы между элементами графика.
    # Делает так, чтобы подписи осей, заголовки и другие элементы не налегали друг на друга и не обрезались.

    plt.show()
    # Показывает все созданные графики в отдельном окне (или выводит их в ноутбуке).
    # Без этой команды график может не отображаться.


In [ ]:
# Базовый путь к изображениям
base_path = r'D:\Семинар 1\png'
# Указываем корневую директорию, в которой хранятся папки с изображениями
# r перед строкой означает "raw" — специальные символы вроде \n не интерпретируются

groups = ['brain', 'goodq_lungs', 'badq_lungs']
# Список подкаталогов (групп изображений), откуда будут загружаться файлы

# Получение списка путей ко всем изображениям
image_paths = {
    group: list(paths.list_images(os.path.join(base_path, group)))
    for group in groups
}
# Для каждой группы:
# - объединяем base_path с именем папки (os.path.join)
# - ищем все изображения внутри с помощью paths.list_images (из imutils)
# - результат сохраняем в словарь, где ключ — имя группы, значение — список путей к изображениям

# Объединение всех путей в один список
all_images = sum(image_paths.values(), [])
# Складываем все списки из словаря в один общий список all_images

# Проверка наличия изображений
if len(all_images) == 0:
    raise ValueError("Ошибка: не найдено ни одного изображения!")
# Если список пуст, вызываем исключение с сообщением об ошибке

# Вывод количества найденных изображений и первых 5 примеров
print(f"Найдено изображений: {len(all_images)}")
print("Примеры путей: ", all_images[:5])
# Показываем общее количество и первые 5 путей к изображениям
# "Постройте гистограммы для всех изображений. Какие вы можете сделать выводы?"

all_histograms = [compute_histogram(p) for p in tqdm(all_images, desc='Расчёт общей гистограммы')]
# Для каждого пути к изображению из списка all_images:
# - вычисляем гистограмму яркости через функцию compute_histogram(p)
# - tqdm добавляет прогресс-бар с подписью 'Расчёт общей гистограммы'
# Результат — список всех гистограмм (одна на изображение)

mean_histogram = np.mean(all_histograms, axis=0)
# Вычисляем среднюю гистограмму по всем изображениям
# axis=0 — считаем среднее по каждому интервалу интенсивности (по столбцам)

plot_histogram(mean_histogram)
# Отображаем среднюю гистограмму с помощью функции визуализации plot_histogram


В реальных условиях при попытке поиска аномалий никаких предзаданных групп, разумеется, не будет,
 но в этом учебном примере снимки заранее разделены на группы
 Постройте гистограммы для каждой группы снимков, сравните их между собой. Какие вы можете сделать выводы?


In [ ]:
for label, paths in image_paths.items():
    # Проходим по каждой группе изображений (словарь: ключ — имя группы, значение — список путей)

    histograms = [compute_histogram(p) for p in paths]
    # Для всех изображений в текущей группе вычисляем гистограммы

    group_mean_hist = np.mean(histograms, axis=0)
    # Вычисляем среднюю гистограмму по группе

    print(f"Гистограмма для группы {label}")
    # Выводим, к какой группе относится текущая гистограмма

    plot_histogram(group_mean_hist)
    # Отображаем гистограмму группы


 Гистограммы явно отличаются, как же нам использовать различия в частоте тех или иных значений для детекции?
 Простейший способ заключается в том, чтобы представить каждую отдельную гистограмму как точку 255-мерного пространства
 и рассчитать расстояние от этих точек до некоторого условного центра, определенного через усредненную гистограмму


In [ ]:
# Обобщенная функция для расчета дистанций. Вспомните материал лекции 1, посвященной мерам дистанции.
def calculate_distance(p, q, metric='euclidean', p_value=2):
    # p, q — два вектора (например, гистограммы изображений)
    # metric — строка, указывающая, какую метрику использовать (по умолчанию 'euclidean')
    # p_value — степень для метрики Минковского (по умолчанию 2)

    if metric == 'euclidean':
        return np.linalg.norm(p - q)
        # Евклидово расстояние: корень из суммы квадратов разностей между компонентами векторов

    elif metric == 'manhattan':
        return np.sum(np.abs(p - q))
        # Манхэттенское расстояние: сумма модулей разностей между элементами векторов

    elif metric == 'cosine':
        return 1 - np.dot(p, q) / (np.linalg.norm(p) * np.linalg.norm(q))
        # Косинусное расстояние: 1 минус косинус угла между векторами
        # np.dot(p, q) — скалярное произведение
        # np.linalg.norm(p) — длина вектора

    elif metric == 'kl_divergence_mod':
        p_safe = np.where(p == 0, 1e-10, p)
        q_safe = np.where(q == 0, 1e-10, q)
        # Заменяем нули на очень маленькое число, чтобы избежать деления на ноль и логарифма нуля

        return np.sum(p_safe * np.log(p_safe / q_safe))
        # Вычисление модифицированной дивергенции Кульбака–Лейблера:
        # мера различия между двумя распределениями вероятностей

    elif metric == 'minkowski':
        return np.linalg.norm(p - q, ord=p_value)
        # Обобщённая метрика Минковского: степень p_value задаёт конкретный тип расстояния
        # p=1 — Манхэттен, p=2 — Евклидово, p=∞ — Чебышевское

    else:
        raise ValueError(f"Unknown metric: {metric}")
        # Ошибка, если передано неизвестное название метрики


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

metrics = ['euclidean', 'manhattan', 'cosine']
# Список метрик расстояния, которые будут использоваться для анализа

x_offsets = np.arange(len(image_paths))
# x-координаты по числу групп (например: 0, 1, 2), чтобы разместить группы по оси X

colors = ['blue', 'green', 'red']
# Цвета для отображения каждой группы

for metric in metrics:
    plt.figure(figsize=(8, 6))
    # Создаём новый график для каждой метрики

    for x, (label, color) in zip(x_offsets, zip(image_paths.keys(), colors)):
        # Перебираем группы изображений: x — координата, label — название группы, color — цвет

        distances = [calculate_distance(compute_histogram(p), mean_histogram, metric)
                     for p in image_paths[label]]
        # Для каждого изображения в группе:
        # - считаем гистограмму
        # - считаем расстояние до средней гистограммы (mean_histogram)
        # - результат — список расстояний

        x_positions = np.full_like(distances, x, dtype=np.float32)
        # Создаём массив координат x, равных номеру группы

        x_positions += np.random.uniform(-0.1, 0.1, size=len(distances))
        # Добавляем небольшое случайное смещение, чтобы точки не накладывались друг на друга

        plt.scatter(x_positions, distances, label=label, c=color, alpha=0.6, edgecolors='k')
        # Отображаем точки на графике (разброс значений для группы)

    plt.xticks(x_offsets, image_paths.keys())  # Подписываем группы по оси X
    plt.title(f"Метрика: {metric}")            # Заголовок графика с названием метрики
    plt.xlabel("Группа")                       # Подпись оси X
    plt.ylabel("Дистанция")                    # Подпись оси Y
    plt.legend()                               # Показываем легенду (названия групп)
    plt.grid(True, linestyle='--', alpha=0.5)  # Включаем сетку
    plt.show()                                 # Показываем график


In [ ]:
import numpy as np                         # Библиотека для работы с массивами и матрицей
import matplotlib.pyplot as plt            # Построение графиков
import itertools                           # Модуль для создания итераторов (например, бесконечный цикл цветов)

# Добавляем больше значений p
p_values = [0.5, 1, 2, 3, 5, 10, 20, 50, 100]
# Значения степени p для метрики Минковского

x_offsets = np.arange(len(image_paths))   # x-координаты для групп изображений (0, 1, 2, ...)

# Расширенная палитра цветов (повторяется при необходимости)
color_palette = ['blue', 'green', 'red', 'purple', 'orange', 'brown', 'pink', 'cyan', 'magenta']
colors = itertools.cycle(color_palette)   # Создаёт бесконечный итератор по цветам (чтобы не закончились)

# Для каждого значения степени p и цвета
for p_value, color in zip(p_values, colors):
    plt.figure(figsize=(8, 6))  # Создаём график

    for x, (label, color) in zip(x_offsets, zip(image_paths.keys(), itertools.cycle(color_palette))):
        # Для каждой группы изображений: x — позиция, label — имя группы, color — цвет

        distances = [calculate_distance(compute_histogram(p), mean_histogram,
                                        metric='minkowski', p_value=p_value)
                     for p in image_paths[label]]
        # Считаем расстояния между гистограммами изображений и средней гистограммой
        # с использованием метрики Минковского

        x_positions = np.full_like(distances, x, dtype=np.float32)
        # Создаём массив координат x для текущей группы

        x_positions += np.random.uniform(-0.1, 0.1, size=len(distances))
        # Добавляем небольшое случайное смещение, чтобы точки не перекрывались

        plt.scatter(x_positions, distances, label=label, c=color, alpha=0.6, edgecolors='k')
        # Отображаем точки на графике

    plt.xticks(x_offsets, image_paths.keys())           # Подписи на оси X — названия групп
    plt.title(f"Минковский p = {p_value}")              # Заголовок с текущим значением p
    plt.xlabel("Группа")                                # Подпись оси X
    plt.ylabel("Дистанция")                             # Подпись оси Y
    plt.legend()                                        # Легенда по группам
    plt.grid(True, linestyle='--', alpha=0.5)           # Сетка на графике
    plt.show()                                          # Показать график


Как можно выделить аномальные изображения на основании вычисленных выше дистанций?

 Насколько эффективным будет такой метод, какая дистанция оказалась наиболее эффективной?

 Попробуйте реализовать расчёт расстояний для логарифмического представления гистограммы,
повысилась ли эффективность?

 Попробуем теперь простейший метод кластеризации k-means и отобразим результаты


In [ ]:
# Превращаем гистограммы в NumPy-массив
all_histograms_flat = np.array(all_histograms)
# Преобразуем список гистограмм (одна на изображение) в двумерный массив: [число_изображений x 256]

# Предположим, что в image_paths.keys() у нас 3–5 групп (пример)
true_labels = np.concatenate([[i] * len(image_paths[label])
                              for i, label in enumerate(image_paths)])
# Формируем массив меток (true_labels) с числами: для каждой группы одна метка (0, 1, 2, ...)
# Например: [0, 0, 0, 1, 1, 2, 2, 2, 2]

# Укажите число кластеров, на которые вы хотите попробовать разделить данные
n_clusters = 3
# Это количество групп, которое мы хотим найти с помощью кластеризации (например, 3)

# Размер "облака" для каждого кластера (радиус) — исключительно параметр отображения
r = 0.3
# r используется только при визуализации, чтобы задать радиус точек на графике


In [ ]:
# Как можно выделить аномальные изображения на основании вычисленных выше дистанций?
# — Здесь предполагается, что ранее были рассчитаны расстояния (например, между гистограммами изображений), и нужно определить, какие изображения "аномальны" — то есть сильно отличаются от остальных.

# Насколько эффективным будет такой метод, какая дистанция оказалась наиболее эффективной?
# — Это вопрос для анализа: нужно сравнить разные метрики расстояний (например, Евклидова, Манхэттенская и т.п.) и выяснить, какая лучше отделяет аномалии.

# Попробуйте реализовать расчет расстояний для логарифмического представления гистограммы, повысилась ли эффективность?
# — Предлагается вместо обычных гистограмм использовать логарифм значений (например, log(1 + значение)), чтобы сгладить влияние пиков и усилить различия в нижнем диапазоне.

# Попробуем теперь простейший метод кластеризации k-mean и отобразим результаты
# — Предлагается применить метод k-средних (k-means) для кластеризации изображений по их признакам (например, гистограммам), и визуализировать результат (например, разными цветами или в отдельных группах).


In [ ]:
# Превращаем гистограммы в NumPy-массив
all_histograms_flat = np.array(all_histograms)
# all_histograms — это список гистограмм изображений
# np.array(...) превращает его в массив NumPy, который удобен для математических операций и подачи в модели

# Предположим, что в image_paths.keys() у нас 3–5 групп изображений (например, по папкам с метками)
true_labels = np.concatenate(
    [[i] * len(image_paths[label])  # создаем список, где число i повторяется столько раз, сколько изображений в данной метке
     for i, label in enumerate(image_paths)]  # перечисляем все метки и их индексы (i — это номер кластера)
)

# Укажите число кластеров, на которые вы хотите попробовать разделить данные
n_clusters = 3  # задаем количество кластеров, например, 3

# Размер "облака" для каждого кластера (радиус) — исключительно для визуализации
R = 0.3

# Инициализируем алгоритм кластеризации k-средних
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
# n_clusters — количество кластеров
# random_state — фиксирует случайность для воспроизводимости результатов
# n_init=10 — алгоритм будет запускаться 10 раз с разными начальными центрами и выберет наилучший

# Обучаем модель и сразу получаем предсказанные метки кластеров
predicted_labels = kmeans.fit_predict(all_histograms_flat)

# Получаем уникальные кластеры из результата кластеризации
unique_clusters = np.unique(predicted_labels)

# Определяем общее число уникальных кластеров (может быть меньше n_clusters, если данные одинаковы)
num_clusters = len(unique_clusters)
# Генерируем случайное расположение центров для каждого кластера
# Чтобы центры не пересекались, выберем прямоугольную область [0, num_clusters] × [0, num_clusters]
# и случайно выберем точку внутри этого прямоугольника для каждого кластера

cluster_centers = {}  # создаем словарь для хранения координат центров кластеров
for cluster in unique_clusters:
    x_c = np.random.uniform(0, num_clusters)  # случайная координата X в пределах [0, num_clusters]
    y_c = np.random.uniform(0, num_clusters)  # случайная координата Y в тех же пределах
    cluster_centers[cluster] = (x_c, y_c)     # сохраняем центр кластера как пару (x, y)

# Создаем массивы для координат всех точек, которые будем отображать
n_points = len(predicted_labels)  # количество точек = количество изображений
x_positions = np.zeros(n_points, dtype=np.float64)  # массив X-координат, инициализируем нулями
y_positions = np.zeros(n_points, dtype=np.float64)  # массив Y-координат, инициализируем нулями

# Заполняем координаты каждой точки на основе ее кластера
for i, cluster in enumerate(predicted_labels):  # проходим по всем точкам и их меткам
    x_c, y_c = cluster_centers[cluster]         # берем центр кластера для текущей точки

    # Генерируем случайную точку в пределах круга радиуса R вокруг центра
    r = np.sqrt(np.random.uniform(0, R**2))     # расстояние от центра (радиальное), равномерно по площади
    theta = np.random.uniform(0, 2 * np.pi)     # угол поворота в радианах (от 0 до 2π)

    dx = r * np.cos(theta)  # смещение по X
    dy = r * np.sin(theta)  # смещение по Y

    x_positions[i] = x_c + dx  # итоговая координата X точки
    y_positions[i] = y_c + dy  # итоговая координата Y точки

# === Отрисовка графика ===
plt.figure(figsize=(8, 6))  # создаем новое изображение размером 8x6 дюймов

# Цвет точек соответствует истинным меткам (true_labels)
scatter = plt.scatter(
    x_positions, y_positions,   # координаты точек
    c=true_labels,              # цвет каждой точки определяется её настоящей меткой
    cmap='viridis',             # используем цветовую карту "viridis"
    edgecolors='k',             # черная обводка точек ('k' = black)
    alpha=0.7                   # прозрачность точек (0 = полностью прозрачные, 1 = непрозрачные)
)

# Подписи кластеров под облаком точек
for cluster in unique_clusters:
    x_c, y_c = cluster_centers[cluster]  # получаем координаты центра кластера

    # plt.text размещает текст на графике
    plt.text(
        x_c, y_c - (R + 0.05),            # размещаем текст немного ниже центра облака
        f"Кластер {cluster}",            # содержимое подписи
        fontsize=10,                     # размер шрифта
        ha='center',                     # горизонтальное выравнивание: по центру
        va='top',                        # вертикальное выравнивание: по верхнему краю
        color='black',                   # цвет текста
        bbox=dict(                       # параметры рамки вокруг текста
            boxstyle="round,pad=0.3",    # скругленная рамка с отступом 0.3
            fc="white",                  # цвет фона рамки (fc = facecolor)
            ec="black",                  # цвет контура рамки (ec = edgecolor)
            alpha=0.5                    # прозрачность рамки
        )
    )
# Легенда для истинных меток (подписи к цветам точек)
plt.legend(
    handles=scatter.legend_elements()[0],  # извлекаем элементы легенды из объекта scatter (только маркеры)
    labels=image_paths.keys(),             # метки соответствуют названиям групп (например, папок с изображениями)
    title="Истинные метки"                 # заголовок для легенды
)

# Заголовок и подписи осей графика
plt.title("Случайное расположение кластеров (цвет = истинные метки)")  # заголовок графика
plt.xlabel("X (случайная координата)")  # подпись оси X
plt.ylabel("Y (случайная координата)")  # подпись оси Y

# Сетка на фоне графика
plt.grid(True, linestyle="--", alpha=0.5)  # включаем сетку, пунктирную, полупрозрачную

# Показываем график
plt.show()
